**Step 1: Import Lib**

In [1]:
import numpy as np
import matplotlib.pylab as plt
from scipy.misc import imresize, imread
import itertools
import sklearn
import cv2
import time
import shutil
import os
from scipy.misc import imresize
from skimage import transform,data
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, Input
from keras.applications.mobilenet import MobileNet
from keras.models import Sequential, Model
%matplotlib inline
plt.style.use({'figure.figsize':(50,40)})

Using TensorFlow backend.


**Step 2: Import Image**

In [2]:
X = cv2.imread('../input/Pic/example1.png')
X1 = cv2.imread('../input/Pic/example1.png')
#X = X.reshape((1,) + X.shape)

In [3]:
"""
plt.imshow(image) # 显示图片(if you want)
plt.axis('on') # 显示坐标轴
plt.show()
"""

"\nplt.imshow(image) # 显示图片(if you want)\nplt.axis('on') # 显示坐标轴\nplt.show()\n"

**Step 3: Identify IDC+ or IDC-**

In [4]:
batch_size = 128
num_classes = 2
epochs = 12
img_rows, img_cols = X.shape[1],X.shape[2]
input_shape = (img_rows, img_cols, 3)
# x_train = a
# y_train = b
# x_test = c
# y_test = d
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

preds = model.predict(X)
T = preds[0][0]
F = preds[0][1]

Instructions for updating:
Colocations handled automatically by placer.


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_2/convolution' (op: 'Conv2D') with input shapes: [?,1358,1,32], [3,3,32,64].

In [15]:
mobilenet_model(input_shape = (1360,1024,3)):
    mobilenet = MobileNet(include_top = False, weights = 'imagenet', input_shape = input_shape)
    
    for layer in mobilenet.layers:
        layer.trainable = False
    last = mobilenet.output
   
    x = Flatten()(last)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4, activation = 'softmax')(x)
    
    model  

In [16]:
model_mobilenet = mobilenet_model()
model_mobilenet.summary()
model_mobilenet.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001), metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1360, 1024, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 1361, 1025, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 680, 512, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 680, 512, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 680, 512, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 680, 512, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 680, 512, 32)      128       
__________

In [17]:
preds = model_mobilenet.predict(X)
T = preds[0][0]
F = preds[0][1]

ValueError: Error when checking input: expected input_2 to have 4 dimensions, but got array with shape (1024, 1360, 3)

**Step 4: Generate Reports**

In [ ]:
if T>F:
    im1 = X1
    Img = cv2.resize(im1, (256,256), interpolation=cv2.INTER_CUBIC)
    kernel_2 = np.ones((2, 2), np.uint8)  # 2x2的卷积核
    kernel_3 = np.ones((3, 3), np.uint8)  # 3x3的卷积核
    kernel_4 = np.ones((4, 4), np.uint8)  # 4x4的卷积核
    if Img is not None:  # 判断图片是否读入
        HSV = cv2.cvtColor(Img, cv2.COLOR_BGR2HSV)  # 把RGB图像转换为HSV格式
        Lower = np.array([90, 25, 0])  # 要识别颜色的下限
        Upper = np.array([154, 255, 225])  # 要识别的颜色的上限
        # mask是把HSV图片中在颜色范围内的区域变成白色，其他区域变成黑色
        mask = cv2.inRange(HSV, Lower, Upper)
        # 下面四行是用卷积进行滤波
        erosion = cv2.erode(mask, kernel_4, iterations=1)
        erosion = cv2.erode(erosion, kernel_4, iterations=1)
        dilation = cv2.dilate(erosion, kernel_4, iterations=1)
        dilation = cv2.dilate(dilation, kernel_4, iterations=1)
        # target是把原图中的非目标颜色区域去掉剩下的图像
        target = cv2.bitwise_and(Img, Img, mask=dilation)
        # 将滤波后的图像变成二值图像放在binary中
        ret, binary = cv2.threshold(dilation, 127, 255, cv2.THRESH_BINARY)
        # 在binary中发现轮廓，轮廓按照面积从小到大排列
        contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        p = 0
        for i in contours:  # 遍历所有的轮廓
            x, y, w, h = cv2.boundingRect(i)  # 将轮廓分解为识别对象的左上角坐标和宽、高
            # 在图像上画上矩形（图片、左上角坐标、右下角坐标、颜色、线条宽度）
            cv2.rectangle(Img, (x, y), (x + w, y + h), (0, 255,), 2)
            # 给识别对象写上标号
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(Img, str(p), (x - 10, y + 10), font, 0.5, (0, 0, 255), 1)  # 加减10是调整字符位置
            p += 1
        print('There are', p, 'potential tumor areas')  # 终端输出目标数量

    plt.figure(figsize=(6, 4))
    # plt.subplot(n_rows, n_cols, plot_num)
    
    plt.subplot(2, 3, 1)
    plt.imshow(X1)
    plt.title('Original')
    plt.axis('off') 
    
    plt.subplot(2, 3, 2)
    plt.imshow(Img)
    plt.title('BC+')
    plt.axis('off') 
    
    plt.subplot(2, 3, 3)
    plt.imshow(target)
    plt.title('Target')
    plt.axis('off') 

    plt.subplot(2, 3, 4)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off') 
    
    plt.subplot(2, 3, 5)
    plt.imshow(dilation)
    plt.title('Prod')
    plt.axis('off') 
    
    plt.show()
else:
    plt.subplot(2, 2, 1)
    plt.imshow(X1)
    plt.title('BC-')
    plt.axis('off') 